### Converting Tensorflow Model to TensorFlow Lite Model

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_datasets as tfds
#tfds.disable_progress_bar()

In [8]:
!pip install tensorflow_hub

  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [10]:
# Convert the model
saved_model_dir = '/root/AI-Projects/Fire_Detection/final_model/1'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel directory
tflite_model = converter.convert()

In [12]:
reloaded = tf.keras.models.load_model(
  saved_model_dir, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 64, 64, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        2

In [28]:
count= 0
for layer in reloaded.layers:
    if((str(layer.name).startswith('batch')) or (str(layer.name).startswith('activ'))):
        print('\n')
    else:
        count+=1
        print(count,layer.name,layer.input_shape)

1 conv2d (None, 128, 128, 3)




2 depthwise_conv2d (None, 64, 64, 32)




3 conv2d_1 (None, 64, 64, 32)




4 depthwise_conv2d_1 (None, 64, 64, 64)




5 conv2d_2 (None, 32, 32, 64)




6 depthwise_conv2d_2 (None, 32, 32, 128)




7 conv2d_3 (None, 32, 32, 128)




8 depthwise_conv2d_3 (None, 32, 32, 128)




9 conv2d_4 (None, 16, 16, 128)




10 depthwise_conv2d_4 (None, 16, 16, 256)




11 conv2d_5 (None, 16, 16, 256)




12 depthwise_conv2d_5 (None, 16, 16, 256)




13 conv2d_6 (None, 8, 8, 256)




14 depthwise_conv2d_6 (None, 8, 8, 512)




15 conv2d_7 (None, 8, 8, 512)




16 depthwise_conv2d_7 (None, 8, 8, 512)




17 conv2d_8 (None, 8, 8, 512)




18 depthwise_conv2d_8 (None, 8, 8, 512)




19 conv2d_9 (None, 8, 8, 512)




20 depthwise_conv2d_9 (None, 8, 8, 512)




21 conv2d_10 (None, 8, 8, 512)




22 depthwise_conv2d_10 (None, 8, 8, 512)




23 conv2d_11 (None, 4, 4, 512)




24 depthwise_conv2d_11 (None, 4, 4, 1024)




25 conv2d_12 (None, 2, 2, 1024)




26 global_averag

In [17]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [18]:
builder = tfds.ImageFolder('s3://swifty-datasets/firenet_data/')
print(builder.info)  # num examples, labels... are automatically calculated
ds = builder.as_dataset(split='train', shuffle_files=True,  as_supervised = True)
tfds.show_examples(ds, builder.info)

NameError: name 'tfds' is not defined

In [ ]:
# Gather results for the randomly sampled test images
predictions = []
test_labels = []
test_images = []

for img, label in ds.take(50):
  interpreter.set_tensor(input_index, img)
  interpreter.invoke()
  predictions.append(interpreter.get_tensor(output_index))
  test_labels.append(label[0])
  test_images.append(np.array(img))

In [ ]:
# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)